In [1]:
!pip install facenet-pytorch

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import precision_recall_curve, average_precision_score
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
from facenet_pytorch import InceptionResnetV1, MTCNN
import torchvision.transforms as transforms

model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

mtcnn = MTCNN(image_size=160, margin=40, device=device)

fallback_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [3]:
def extract_embedding(img_path):
    img = Image.open(img_path).convert('RGB')

    face_tensor = mtcnn(img)

    if face_tensor is None:
        face_tensor = fallback_transform(img)

    with torch.no_grad():
        face_tensor = face_tensor.unsqueeze(0).to(device)
        embedding = model(face_tensor)
        embedding = F.normalize(embedding, p=2, dim=1)

    return embedding.cpu().numpy().flatten()

def cosine_similarity(emb1, emb2):
    return np.dot(emb1, emb2)

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Test Dataset.zip to Test Dataset.zip


In [8]:
!unzip -q -o '/content/Test Dataset.zip' -d '/content/'

In [9]:
gallery_ids = ['00', '01', '02', '03', '04', '05', '06']
gallery = {}
gallery_images = {}

print("Building gallery embeddings...")
for person_id in gallery_ids:
    folder_path = f'/content/Test Dataset/{person_id}'
    embeddings = []
    first_img = None

    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        emb = extract_embedding(img_path)
        embeddings.append(emb)
        if first_img is None:
            first_img = Image.open(img_path)

    gallery[person_id] = np.array(embeddings)
    gallery_images[person_id] = first_img

queries_path = '/content/Test Dataset/queries'
query_files = sorted(os.listdir(queries_path))

Building gallery embeddings...


In [10]:
def attention_score(query_emb, gallery_embs, temperature=1.0):
    sims = np.array([cosine_similarity(query_emb, g) for g in gallery_embs])

    exp_sims = np.exp(sims / temperature)
    attention = exp_sims / np.sum(exp_sims)

    S_p = np.sum(attention * sims)

    return S_p, attention, sims

def max_score(query_emb, gallery_embs):
    sims = np.array([cosine_similarity(query_emb, g) for g in gallery_embs])
    return np.max(sims), sims

def avg_score(query_emb, gallery_embs):
    sims = np.array([cosine_similarity(query_emb, g) for g in gallery_embs])
    return np.mean(sims), sims

In [11]:
results = []

for query_name in query_files:
    query_path = os.path.join(queries_path, query_name)
    query_emb = extract_embedding(query_path)

    ground_truth = query_name.split('_')[1].split('.')[0]
    is_known = ground_truth in gallery_ids

    scores = {}
    for person_id, gallery_embs in gallery.items():
        S_p, attention, pose_sims = attention_score(query_emb, gallery_embs)
        scores[person_id] = {
            'attention_score': S_p,
            'max_score': np.max(pose_sims),
            'avg_score': np.mean(pose_sims),
            'attention_weights': attention,
            'pose_sims': pose_sims
        }

    predicted = max(scores.keys(), key=lambda p: scores[p]['attention_score'])
    best_score = scores[predicted]['attention_score']

    results.append({
        'query': query_name,
        'ground_truth': ground_truth,
        'is_known': is_known,
        'predicted': predicted,
        'score': best_score,
        'all_scores': {p: s['attention_score'] for p, s in scores.items()},
        'correct_id': (predicted == ground_truth) if is_known else None
    })

print(f"Processed {len(results)} queries")

Processed 42 queries


In [12]:
known_results = [r for r in results if r['is_known']]
unknown_results = [r for r in results if not r['is_known']]

correct = sum(1 for r in known_results if r['correct_id'])
total = len(known_results)

print("="*60)
print("IDENTIFICATION RESULTS")
print("="*60)
print(f"Known people (00-06): {correct}/{total} = {100*correct/total:.1f}%")
print(f"Unknown people (07): {len(unknown_results)} queries")
print("="*60)

print("\nDetailed Results (Known):")
for r in known_results:
    status = "✓" if r['correct_id'] else "✗"
    print(f"{status} {r['query']} → Pred: {r['predicted']}, Truth: {r['ground_truth']}, Score: {r['score']:.3f}")

print("\nUnknown queries (07):")
for r in unknown_results:
    print(f"  {r['query']} → Pred: {r['predicted']}, Score: {r['score']:.3f}")

IDENTIFICATION RESULTS
Known people (00-06): 27/29 = 93.1%
Unknown people (07): 13 queries

Detailed Results (Known):
✓ 01_00.jpg → Pred: 00, Truth: 00, Score: 0.643
✓ 04_01.jpg → Pred: 01, Truth: 01, Score: 0.679
✓ 05_02.jpg → Pred: 02, Truth: 02, Score: 0.664
✓ 07_03.jpg → Pred: 03, Truth: 03, Score: 0.682
✓ 08_00.jpg → Pred: 00, Truth: 00, Score: 0.593
✓ 11_03.jpg → Pred: 03, Truth: 03, Score: 0.775
✓ 13_04.jpg → Pred: 04, Truth: 04, Score: 0.557
✗ 16_04.jpg → Pred: 05, Truth: 04, Score: 0.508
✓ 17_03.jpg → Pred: 03, Truth: 03, Score: 0.722
✓ 18_01.jpg → Pred: 01, Truth: 01, Score: 0.628
✓ 19_05.jpg → Pred: 05, Truth: 05, Score: 0.604
✓ 20_00.jpg → Pred: 00, Truth: 00, Score: 0.564
✓ 21_03.jpg → Pred: 03, Truth: 03, Score: 0.620
✓ 22_05.jpg → Pred: 05, Truth: 05, Score: 0.706
✓ 23_00.jpg → Pred: 00, Truth: 00, Score: 0.610
✓ 24_03.jpg → Pred: 03, Truth: 03, Score: 0.539
✓ 26_03.jpg → Pred: 03, Truth: 03, Score: 0.818
✓ 28_04.jpg → Pred: 04, Truth: 04, Score: 0.626
✓ 29_01.jpg → Pred